In [1]:
!pip install wget
!pip install ftfy

In [2]:
%load_ext autoreload
%autoreload 2

from google.colab import drive

drive.mount("/content/drive")
import os
import sys

# TODO: Fill in the Google Drive path where you uploaded the assignment
# Example: If you create a WI2022 folder and put all the files under A5 folder, then "WI2022/A5"
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = './eecs545pro'
GOOGLE_DRIVE_PATH = os.path.join("drive", "My Drive", GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))


# Add to sys so we can import .py files.
sys.path.append(GOOGLE_DRIVE_PATH)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['a4_helper.py', 'simple_tokenizer.py', 'clip.py', 'bpe_simple_vocab_16e6.txt.gz', '__init__.py', 'model.py', 'common.py', 'FPN_ROI.py', 'clip', 'A4', 'mAP', 'eecs598', '__pycache__', 'FPN_ROI align.ipynb', 'voc07_val.json', 'voc07_train.json']


In [3]:
from a4_helper import *
from eecs598 import reset_seed
import multiprocessing


# Set a few constants related to data loading.
NUM_CLASSES = 20
BATCH_SIZE = 16
IMAGE_SHAPE = (224*4, 224*4)
NUM_WORKERS = multiprocessing.cpu_count()
from a4_helper import VOC2007DetectionTiny

train_dataset = VOC2007DetectionTiny(
    GOOGLE_DRIVE_PATH_AFTER_MYDRIVE, "train", image_size=IMAGE_SHAPE[0],
    download=False  # True (for the first time)
)
val_dataset = VOC2007DetectionTiny(GOOGLE_DRIVE_PATH_AFTER_MYDRIVE, "val", image_size=IMAGE_SHAPE[0])
DEVICE = 'cpu'

print(train_dataset)

In [4]:
import torch
from common import DetectorBackboneWithFPN
import torchvision


# sanity check
backbone = DetectorBackboneWithFPN(out_channels=64)

dummy_images = torch.randn(2, 3, 224, 224)

dummy_fpn_feats = backbone(dummy_images)

For dummy input images with shape: (2, 3, 224, 224)
Shape of p3 features: torch.Size([2, 64, 28, 28])
Shape of p4 features: torch.Size([2, 64, 14, 14])
Shape of p5 features: torch.Size([2, 64, 7, 7])
Shape of p6 features: torch.Size([2, 64, 4, 4])
Shape of p7 features: torch.Size([2, 64, 2, 2])


In [5]:
# # model block sanity check
# model = torchvision.models.regnet_x_400mf(pretrained=True)
# train_nodes, eval_nodes = torchvision.models.feature_extraction.get_graph_node_names(model)

# print(train_nodes)
# print(eval_nodes)

In [6]:
# FPN trainability check

# from torch import nn
# from a4_helper import train_detector
# from common import DetectorBackboneWithFPN
# from FPN_ROI import RPN
# reset_seed(0)

# # Take equally spaced examples from training dataset to make a subset.
# small_dataset = torch.utils.data.Subset(
#     train_dataset,
#     torch.linspace(0, len(train_dataset) - 1, steps=BATCH_SIZE * 10).long()
# )
# small_train_loader = torch.utils.data.DataLoader(
#     small_dataset, batch_size=BATCH_SIZE, pin_memory=True
# )

# # Create a wrapper module to contain backbone + RPN:
# class FirstStage(nn.Module):
#     def __init__(self, fpn_channels: int):
#         super().__init__()
#         self.backbone = DetectorBackboneWithFPN(out_channels=fpn_channels)
#         self.rpn = RPN(
#             fpn_channels=fpn_channels,
#             # Simple stem of two layers:
#             stem_channels=[fpn_channels, fpn_channels],
#             batch_size_per_image=16,
#             anchor_stride_scale=8,
#             anchor_aspect_ratios=[0.5, 1.0, 2.0],
#             anchor_iou_thresholds=(0.3, 0.6),
#         )

#     def forward(self, images, gt_boxes=None):
#         feats_per_fpn_level = self.backbone(images)
#         return self.rpn(feats_per_fpn_level, self.backbone.fpn_strides, gt_boxes)


# first_stage = FirstStage(fpn_channels=64).to(DEVICE)

# train_detector(
#     first_stage,
#     small_train_loader,
#     learning_rate=8e-3,
#     max_iters=1000,
#     log_period=20,
#     device=DEVICE,
# )

In [7]:
# load data

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=BATCH_SIZE, pin_memory=True
)

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=1, pin_memory=True
)

train_loader_iter = iter(train_loader)
image_paths, images, gt_boxes = train_loader_iter.next()

# print(f"image paths           : {image_paths}")
print(f"image batch has shape : {images.shape}")
print(f"gt_boxes has shape    : {gt_boxes.shape}")

print(f"Five boxes per image  :")
print(gt_boxes[:, :5, :])
print(images[0,0,:,:])


image batch has shape : torch.Size([16, 3, 896, 896])
gt_boxes has shape    : torch.Size([16, 40, 5])
Five boxes per image  :
tensor([[[195.7477, 260.9970, 720.4324, 726.4865,   6.0000],
         [ -1.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000],
         [ -1.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000],
         [ -1.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000],
         [ -1.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000]],

        [[313.3846, 152.6154, 544.7692, 489.8461,  14.0000],
         [ 79.5385, 192.0000, 850.0000, 827.0769,  12.0000],
         [ -1.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000],
         [ -1.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000],
         [ -1.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000]],

        [[ 24.1437, 395.0060, 657.2455, 896.0000,   1.0000],
         [617.0060, 368.1796, 896.0000, 896.0000,   1.0000],
         [  5.3653,   0.0000, 313.8683, 767.8922,  14.0000],
         [  8.0479,   0.0000, 651.8802, 896.0000,  14.0000],
         [603.59

In [ ]:
from torch import nn

from a4_helper import train_detector
from common import DetectorBackboneWithFPN
from FPN_ROI import RPN
import clip

reset_seed(0)
from FPN_ROI import FasterRCNN

model, preprocess = clip.load("ViT-B/32")
# Slightly larger detector than in above cell.
FPN_CHANNELS = 128
backbone = DetectorBackboneWithFPN(out_channels=FPN_CHANNELS)
rpn = RPN(
    fpn_channels=FPN_CHANNELS,
    stem_channels=[FPN_CHANNELS, FPN_CHANNELS],
    batch_size_per_image=16,
    pre_nms_topk=500,
    post_nms_topk=200  # Other args from previous cell are default args in RPN.
)
# fmt: off
faster_rcnn = FasterRCNN(
    backbone, rpn, num_classes=NUM_CLASSES, roi_size=(7, 7),
    stem_channels=[FPN_CHANNELS, FPN_CHANNELS],
    batch_size_per_image=32,
    clip_model = model
)
# fmt: on

train_detector(
    faster_rcnn,
    train_loader,
    learning_rate=0.01,
    max_iters=9000,
    log_period=50,
    device=DEVICE,
)

# After you've trained your model, save the weights for submission.
weights_path = os.path.join('./A4', "rcnn_detector.pt")
torch.save(faster_rcnn.state_dict(), weights_path)

For dummy input images with shape: (2, 3, 224, 224)
Shape of p3 features: torch.Size([16, 128, 112, 112])
Shape of p4 features: torch.Size([16, 128, 56, 56])
Shape of p5 features: torch.Size([16, 128, 28, 28])
Shape of p6 features: torch.Size([16, 128, 14, 14])
Shape of p7 features: torch.Size([16, 128, 7, 7])
dict_keys(['p3', 'p4', 'p5', 'p6', 'p7'])
torch.Size([33, 3, 224, 224])
torch.Size([33, 512])
[Iter 0][loss: 3.970][loss_rpn_obj: 0.694][loss_rpn_box: 0.238][loss_cls: 3.038]
For dummy input images with shape: (2, 3, 224, 224)
Shape of p3 features: torch.Size([16, 128, 112, 112])
Shape of p4 features: torch.Size([16, 128, 56, 56])
Shape of p5 features: torch.Size([16, 128, 28, 28])
Shape of p6 features: torch.Size([16, 128, 14, 14])
Shape of p7 features: torch.Size([16, 128, 7, 7])
dict_keys(['p3', 'p4', 'p5', 'p6', 'p7'])
torch.Size([41, 3, 224, 224])
torch.Size([41, 512])
For dummy input images with shape: (2, 3, 224, 224)
Shape of p3 features: torch.Size([16, 128, 112, 112])
S

In [ ]:
import clip
import torch

clip.available_models()
model, preprocess = clip.load("ViT-B/32")
cropped_images_all = torch.randn(33,3,224*4,224).to('cuda')
# print(cropped_images_all)
model.encode_image(cropped_images_all).float()